In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
len(os.listdir('/content/drive/MyDrive/Cloudphysician/datasets/yolo unlabbeled dataset/dataset/train/images'))

9000

In [ ]:
from tqdm import tqdm

In [ ]:
!pip install ultralytics
!pip install -U albumentations

In [ ]:
import cv2 as cv 
import os
import albumentations as A
import torch
from ultralytics import YOLO

In [ ]:
os.listdir(non_aug_folder)

In [ ]:
Teacher=YOLO(Teacher_model_path)
Student=YOLO(Student_model_path)

In [ ]:
Teacher_model_path='/content/drive/MyDrive/Cloudphysician/weights for yolov8 /best_annotatedv8s.pt'
Student_model_path='/content/drive/MyDrive/Cloudphysician/weights for yolov8 /best_annotatedv8s.pt'
non_aug_folder='/content/drive/MyDrive/Cloudphysician/datasets/yolo unlabbeled dataset/dataset/train/non_aug'
data_yaml='/content/drive/MyDrive/Cloudphysician/datasets/yolo unlabbeled dataset/dataset/data.yaml'
aug_folder='/content/drive/MyDrive/Cloudphysician/datasets/yolo unlabbeled dataset/dataset/train/images'
label_folder='/content/drive/MyDrive/Cloudphysician/datasets/yolo unlabbeled dataset/dataset/train/labels'
Fake_dataset='/content/drive/MyDrive/Cloudphysician/datasets/yolo unlabbeled dataset/dummy_dataset/data.yaml'

In [ ]:
def Teacher_relabel(images_path,labels_path,Teacher,threshold=0):
  if type(Teacher) == str:Teacher=YOLO(Teacher)##if string is passed
  list_images=os.listdir(images_path)
  for images in tqdm(list_images):
    try:
      result = Teacher(images_path+'/'+images)
      result=result[0].cpu().numpy()
      with open(labels_path+'/'+images[:-4]+'txt','w') as f:
        for boundingbox in result.boxes:
            if boundingbox.conf[0]>threshold:
              f.write(str(int(boundingbox.cls[0]))+' ')
              f.write(str(list(boundingbox.xywhn[0]))[1:-1].replace(',',' ')+'\n')
    except:
      print('not valid image')

In [ ]:
def transfer(src, dest):
  listimgs= os.listdir(src);
  for i in tqdm(listimgs):
    try:
      images = (cv.imread(os.path.join(src, i)))
      transform = A.Compose([
          A.PixelDropout(0.03,True,p=0.7),
          A.ColorJitter (brightness=0.4, contrast=0.4, saturation=0.9, hue=0.5, p=0.6),
          A.GaussNoise (var_limit=(10.0, 50.0), mean=5, per_channel=True, p=0.6),
          A.Downscale(scale_min=0.5, scale_max=0.5,interpolation=cv.INTER_NEAREST, p=0.6)
      ])
      images=transform(image=images)["image"]
      path= os.path.join(dest, i)
      if(os.path.exists(dest)):
        cv.imwrite(path, images)
      else:
        os.makedirs(dest)
        cv.imwrite(path, images)
    except:
      print('not valid image')
    

In [ ]:
def Train_children(yaml_file,student,epochs):
  if type(student)==str: student =YOLO(student)
  student.train(data=yaml_file, epochs=epochs, imgsz=384,augment=False)

In [ ]:
def change_weights(Teacher,Student,alp):
        for (teacher_layer,teacher_value), (student_layer,student_value) in zip(Teacher.model.named_parameters(),Student.model.named_parameters()):
            values = weight_proc(teacher_value,student_value,alp)
            teacher_value.data = values

In [ ]:
def weight_proc(x,y,alp):
  #define a wight manipualting function
  return torch.mul(x,alp).cuda()+torch.mul(y,1-alp).cuda()


In [ ]:
Teacher.to('cuda')
Student.to('cuda')
for i in tqdm(range(15)):
  %time Teacher_relabel(non_aug_folder,label_folder,Teacher,0.7)
  %time  transfer(non_aug_folder,aug_folder)
  %time  Train_children(data_yaml,Student,10)
  %time  Train_children(Fake_dataset,Teacher,1)
  %time  Teacher.to('cuda')
  %time  Student.to('cuda')
  %time  change_weights(Teacher,Student,0.95)